In [4]:
#!pip install Pillow
#!pip install textwrap3
#!pip install json
#!pip install urllib
#!pip install bs4
#!pip install requests
#!pip install babel

In [5]:
from PIL import Image, ImageDraw, ImageFont
import json
from urllib.request import urlopen
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import re
from textwrap3 import wrap
from babel.numbers import format_currency
import decimal
import os

In [6]:
layout = Image.open("maket_plain.jpg") # макет должен лежать в той же папке, что и скрипт

In [7]:
base_config = json.load(open("base_config.json", 'r'))

In [8]:
def feedParser(fpath: str):
    
    # работает и со ссылками, и с локальными файлами    
    if fpath.startswith("http"):
        xmlSoup = BeautifulSoup(urlopen(fpath), "xml")
    else:
        xmlSoup = BeautifulSoup(open(fpath, 'r', encoding='utf-8'), "xml") 
        
    feedDict = dict()    
    
    for item in xmlSoup.find_all("item"):
        itemID = item.find("g:id").text
        iTitle = item.find("g:title").text
        iImageLink = item.find("g:image_link").text.strip()
        iBrand = item.find("g:brand").text.strip()
        iPrice = item.find("g:price").text.strip()
        iSalePrice = item.find("g:sale_price").text.strip() if item.find("g:sale_price") else ""
        iSaleDate = item.find("g:sale_price_effective_date").text.strip() if item.find("g:sale_price_effective_date") else ""
        iAvailability = item.find("g:availability").text.strip()
        iWeight = item.find("g:shipping_weight").text.strip()
        iProductType = item.find("g:product_type").text.strip()
        
        feedDict[itemID] = {
            "ID": itemID,
            "title": iTitle,
            "image": iImageLink,
            "brand": iBrand,
            "price": iPrice,
            "sale_price": iSalePrice,
            'sale_price_effective_date': iSaleDate,
            "shipping_weight": iWeight,
            "product_type": iProductType,
            "availability": iAvailability
            
        }
    
    return feedDict

In [9]:
def attach_image(confDict: dict, item: dict, layout):
    if confDict["content"] == "feed":
        response = requests.get(item["image"])
        img = Image.open(BytesIO(response.content))
    else:
        fpath = confDict["content"]
        if fpath.startswith("http") or fpath.startswith("www"):
            response = requests.get(fpath)
            img = Image.open(BytesIO(response.content))
        else:
            img = Image.open(fpath)
    if "width" in confDict:
        width = confDict['width']
        raw_size = img.size
        aspect_ratio = raw_size[1] / raw_size[0]
        height = round(aspect_ratio * width)
        img = img.resize((width, height), Image.ANTIALIAS)
    
    margin_top = confDict["margin-top"]
    margin_left = confDict["margin-left"]
    
    layout.paste(img, (margin_left, margin_top, margin_left+img.width, margin_top+img.height))

In [10]:
def price_customer(price):
    price = price.strip()
    amount, curr = price.rsplit(' ', maxsplit=1)
    
    if amount.split('.')[1] == "00":
        return format_currency(decimal.Decimal(amount), curr, locale='en_US')[:-3]
    else:    
        return format_currency(decimal.Decimal(amount), curr, locale='en_US')

In [11]:
def attach_text(confDict: dict, item: dict, layout):
    content = confDict["content"]
    if content in item:
        text = item[content]
    else:
        text = content
        
    if content == "price" or content == "sale_price":
        text = price_customer(text)
        
    size = confDict["font-size"]
    font_path = confDict["font"]
    align = confDict["align"]
    #line_height = confDict["line_height"]
    
    font = ImageFont.truetype(open(font_path, "rb"), size=size)
    
    margin_top = confDict["margin-top"]
    margin_left = confDict["margin-left"]
    
    if "width" in confDict:
        width = confDict["width"]
    else:
        width = 40
    line_height = confDict["line-height"]
    
    draw_text = ImageDraw.Draw(layout)
    
    for line in wrap(text, width):
        draw_text.text((margin_left, margin_top), line, font=font, align=align, fill='#1C0606')
        margin_top += line_height

In [15]:
def produceCreative(item: dict, base_layout=layout, base_config=base_config):
    layout = base_layout.copy()
    
    for feature in base_config:
        if feature.startswith("text"):
            attach_text(base_config[feature], item=item, layout=layout)
        elif feature.startswith("image"):
            attach_image(base_config[feature], item=item, layout=layout)
    
    layout.save(os.path.join(os.getcwd(), "output", item["ID"] + ".jpg"))
    layout.close()

In [13]:
feed = feedParser("scanlantheodore.myshopify.xml") # don't forget to write the url

In [ ]:
for picID in list(feed.keys()):
    produceCreative(feed[picID])
    #print(picID)